In [6]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Parameters
using Reel
using StaticArrays
using ProfileView

using Revise

In [7]:
include("../src/pedestrian_crossing/emergency_system.jl")

getObjectConfidenceInterval (generic function with 1 method)

In [8]:
function AutomotiveDrivingModels.observe!(model::EmergencySystem, scene::Scene, roadway::Roadway, egoid::Int)


    model.t_current = model.t_current + model.timestep    
    ego = scene[findfirst(scene, egoid)]

    model.sensor_observations = measure(model.sensor, ego, scene, roadway, model.obstacles)

    (ttb, stop_distance) = getStopDistanceVehicle(model, ego.state.v, 10.0, 1.0)
    print("TTB: ", ttb)
    
    model.risk = 0
    model.collision_rate = 0
    model.brake_request = false
    model.prediction = []

    brake_request = false
    a = 0.0; phi_dot = 0.0
    t0 = 0.0; x0 = 0.0; y0 = 0.0; phi0 = 0.0
    TS = 0.01; T = 3.0
    
    ego_data = caclulateCTRAModel(t0, x0, y0, phi0, ego.state.v, phi_dot, a, TS, T)
    for object in model.sensor_observations
      #  println(object)
        object_posF = Frenet(object.state.posG, get_lane(env.roadway, ego.state), env.roadway)
        
        delta_s = object_posF.s - ego.state.posF.s - ego.def.length/2
        delta_d = object_posF.t - ego.state.posF.t
        delta_theta = object_posF.ϕ - ego.state.posF.ϕ
        #ttc = delta_s / ego.v
        ped_v = object.state.v
        if (delta_s > 0)
            
            # in general, trajectory should come from a prediction model!!
            object_data = caclulateCTRAPredictionSimple(t0, delta_s, 0.1, delta_d, 0.1, 
                                                            delta_theta, 0.1, ped_v, 0.1, TS, T)

            (idx, ti) = getConflictZone(ego_data, object_data)
            (ttc, model.collision_rate, prediction) = caluclateCollision(ego_data, object_data, idx)
            model.prediction = prediction

            ttc_m = mean(ttc)
            ttc_std = std(ttc)

            print(" collision_rate: ", model.collision_rate, " ttc_m: ", ttc_m, " ttc_std: ", ttc_std)
         #   println("ttc_min: ", ttc_m-ttc_std)
            model.ttc = ttc_m-ttc_std
            model.risk = min(ttb / model.ttc, 1.0)
            println("Risk: ", model.risk, " ttc_min: ", model.ttc)

            if ( model.risk > 0.99 && model.collision_rate >= 0.6 )
                brake_request = true
         #       println("Brake request!!!")
            end
        end
    end
    
    if ( brake_request == true )
        model.brake_request = true
    else
        model.brake_request = false
    end
    
    # brake system
    model = brakeSystemStateMachine(model, ego.state)
    #model.a_current = 0
    model.a = LatLonAccel(0.0, model.a_current)
  #  println("model_state: ", model.state, " ", model.a) 
end


In [9]:
params = CrosswalkParams()
params.obstacles_visible = true

obstacle_1 = ConvexPolygon([VecE2(15, -1.5), VecE2(15, -4.5), VecE2(21.5, -4.5), VecE2(21.5, -1.5)],4)
obstacle_2 = ConvexPolygon([VecE2(15, +4.5), VecE2(15, +7.5), VecE2(21.5, +7.5), VecE2(21.5, +4.5)],4)
params.obstacles = [obstacle_1, obstacle_2]
#params.obstacles = [obstacle_2]

env = CrosswalkEnv(params);




ego_v = 10.0
ped_v = 5.0/3.6
hitpoint = 100             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 25.5
ped_y_start = -2
ego_y = 0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;


# Car definition
car_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
car = Vehicle(car_initial_state, VehicleDef(), 1)

# Pedestrian definition using our new Vehicle definition
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

scene = Scene()
push!(scene, car)
push!(scene, ped)



cam = FitToContentCamera(0.);
timestep = 0.05


pos_noise = 0.1
vel_noise = 0.1
false_positive_rate = 0.05
false_negative_rate = 0.01
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 


# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()

ego_id = 1
ped_id = 2
models[ego_id] = EmergencySystem(a=LatLonAccel(0.0, 0.0),env=env,sensor=sensor,obstacles=env.obstacles, timestep=timestep)
models[ped_id] = ConstantPedestrian(v_desired=ped_v, dawdling_amp=0.05) # dumb model


nticks = 80
rec = SceneRecord(nticks+1, timestep)
# execute the simulation

risk = Float64[]
collision_rate = Float64[]
ttc = Float64[]
brake_request = Bool[]
prediction = Vector{Array{Float64}}()

sensor_observations = [Vehicle[]]

obs_callback = (ObservationCallback(risk, collision_rate, ttc, brake_request,prediction,sensor_observations),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()




TTB: 0.885standby
TTB: 0.885standby
TTB: 0.885standby
TTB: 0.885standby
TTB: 0.885 collision_rate: 0.4 ttc_m: 1.9323877837371009 ttc_std: 0.05909008244056107Risk: 0.4724289147354834 ttc_min: 1.8732977012965397
standby
TTB: 0.885 collision_rate: 0.8 ttc_m: 1.8733009098007467 ttc_std: 0.06650055623679876Risk: 0.489816153873515 ttc_min: 1.8068003535639479
standby
TTB: 0.885 collision_rate: 0.4 ttc_m: 1.8114532031557358 ttc_std: 0.05660207896937971Risk: 0.5043162851836414 ttc_min: 1.754851124186356
standby
TTB: 0.885 collision_rate: 0.4 ttc_m: 1.7675678872675378 ttc_std: 0.05556541085638736Risk: 0.5169385045839505 ttc_min: 1.7120024764111503
standby
TTB: 0.885 collision_rate: 1.0 ttc_m: 1.7062814035546914 ttc_std: 0.05708269629929664Risk: 0.5366242382476892 ttc_min: 1.6491987072553946
standby
TTB: 0.885 collision_rate: 0.6 ttc_m: 1.6413979102299514 ttc_std: 0.06305146789936499Risk: 0.5607133999638311 ttc_min: 1.5783464423305864
standby
TTB: 0.885 collision_rate: 0.4 ttc_m: 1.61616930473430

1.206611605

In [10]:
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, prediction)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpUGYCN6", 0x0000000000000050, 20.0, nothing)